# 🔄 PyTorch 自动求导机制 (Autograd) 深度解析

## 📚 本节学习目标
- 深入理解自动求导的原理和重要性
- 掌握计算图的概念和构建过程
- 熟练使用 `requires_grad` 和 `backward()`
- 理解梯度的计算、存储和清零
- 掌握链式法则在自动求导中的应用
- 学会处理复杂的梯度计算场景
- 了解自动求导的高级特性和注意事项

---

## 🤔 为什么需要自动求导？

在深度学习中，我们需要计算损失函数对模型参数的梯度来更新参数。手动计算梯度既容易出错又极其复杂，尤其是在深层神经网络中。

**自动求导的优势：**
- 🎯 **自动化**：无需手动推导复杂的梯度公式
- ✅ **准确性**：避免人为计算错误
- 🚀 **效率**：优化的计算图执行
- 🔧 **灵活性**：支持动态计算图

---

In [ ]:
# 导入必要的库
import torch
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# 添加项目根目录到路径
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)

from utils.common import get_device, print_tensor_info, set_seed

# 设置随机种子确保结果可重现
set_seed(42)

print(f"🔥 PyTorch 版本: {torch.__version__}")
device = get_device()

# 设置matplotlib中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 🔍 1. 自动求导基础概念

### 1.1 什么是计算图？

**计算图(Computational Graph)** 是一个有向无环图，用来表示数学运算的执行过程：
- **节点**：表示变量或运算
- **边**：表示数据流向
- **前向传播**：从输入到输出计算结果
- **反向传播**：从输出到输入计算梯度

### 🌰 举个生活化的例子
假设你在做菜，计算总成本：
```
蔬菜价格(a) = 10元    肉价格(b) = 20元
    ↓                   ↓
  蔬菜成本 = a × 2    肉成本 = b × 1.5
    ↓                   ↓
      总成本(c) = 蔬菜成本 + 肉成本
```

如果蔬菜价格变化1元，总成本变化多少？这就是求梯度 ∂c/∂a！

In [ ]:
print("🔍 自动求导基础演示")
print("=" * 50)

# 创建需要梯度的张量 - 这是关键！
# requires_grad=True 告诉PyTorch："我需要计算这个张量的梯度"
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

print(f"📊 输入变量:")
print(f"x = {x}, requires_grad = {x.requires_grad}")
print(f"y = {y}, requires_grad = {y.requires_grad}")
print()

# 定义一个简单的计算：z = x² + 2xy + y²
# 这相当于 z = (x + y)²
z = x**2 + 2*x*y + y**2

print(f"🧮 计算过程:")
print(f"z = x² + 2xy + y² = {x}² + 2×{x}×{y} + {y}² = {z}")
print(f"z.requires_grad = {z.requires_grad}  # 自动继承")
print(f"z.grad_fn = {z.grad_fn}  # 记录如何计算得到z")
print()

# 计算梯度 - 反向传播！
print(f"🔄 开始反向传播...")
z.backward()  # 计算 dz/dx 和 dz/dy

print(f"✅ 梯度计算完成!")
print(f"dz/dx = {x.grad}  # 理论值: 2x + 2y = 2×{x.item()} + 2×{y.item()} = {2*x.item() + 2*y.item()}")
print(f"dz/dy = {y.grad}  # 理论值: 2y + 2x = 2×{y.item()} + 2×{x.item()} = {2*y.item() + 2*x.item()}")

print("\n💡 观察：自动求导的结果与理论计算完全一致！")

### 1.2 requires_grad 的作用机制

In [ ]:
print("🔧 requires_grad 详解")
print("=" * 40)

# 1. 默认情况：不需要梯度
a = torch.tensor(5.0)  # requires_grad=False (默认)
b = torch.tensor(3.0, requires_grad=True)  # 显式设置为True

print(f"📊 基础张量:")
print(f"a = {a}, requires_grad = {a.requires_grad}")
print(f"b = {b}, requires_grad = {b.requires_grad}")
print()

# 2. 传播规则：只要有一个操作数需要梯度，结果就需要梯度
c = a + b  # a不需要梯度，b需要梯度 → c需要梯度
d = a * 2  # a不需要梯度，常数运算 → d不需要梯度

print(f"🔄 传播规则演示:")
print(f"c = a + b = {c}, requires_grad = {c.requires_grad}  # 继承自b")
print(f"d = a * 2 = {d}, requires_grad = {d.requires_grad}  # a不需要梯度")
print()

# 3. 动态控制：可以随时开启/关闭梯度计算
print(f"🎛️ 动态控制梯度:")

# 开启梯度
a.requires_grad_(True)  # 注意：带下划线的方法是就地操作
print(f"开启后 a.requires_grad = {a.requires_grad}")

# 关闭梯度
b.requires_grad_(False)
print(f"关闭后 b.requires_grad = {b.requires_grad}")

# 4. 只有浮点数类型才能计算梯度
try:
    int_tensor = torch.tensor(5, requires_grad=True)  # 整数类型
except RuntimeError as e:
    print(f"\n❌ 错误: {e}")
    print("💡 解决方案: 使用 .float() 转换为浮点类型")
    int_tensor = torch.tensor(5, dtype=torch.float32, requires_grad=True)
    print(f"✅ 修正后: {int_tensor}, dtype = {int_tensor.dtype}")

## 🧠 2. 计算图的构建和执行

PyTorch使用**动态计算图**，这意味着图是在运行时构建的，每次前向传播都会重新构建图。

In [ ]:
print("🧠 计算图构建过程详解")
print("=" * 50)

# 创建输入节点
x = torch.tensor(2.0, requires_grad=True)
print(f"🎯 步骤1: 创建输入节点")
print(f"x = {x}")
print(f"x.grad_fn = {x.grad_fn}  # 输入节点没有grad_fn")
print()

# 逐步构建计算图
print(f"🔨 步骤2: 逐步构建计算图")

# 第一步：x² 
x_squared = x ** 2
print(f"y1 = x² = {x_squared}")
print(f"y1.grad_fn = {x_squared.grad_fn}  # 记录这是幂运算")

# 第二步：2x²
two_x_squared = 2 * x_squared 
print(f"y2 = 2×y1 = {two_x_squared}")
print(f"y2.grad_fn = {two_x_squared.grad_fn}  # 记录这是乘法运算")

# 第三步：2x² + 1
final_result = two_x_squared + 1
print(f"y3 = y2 + 1 = {final_result}")
print(f"y3.grad_fn = {final_result.grad_fn}  # 记录这是加法运算")
print()

# 查看计算图的next_functions
print(f"🔍 计算图连接关系:")
print(f"final_result 的前一个函数: {final_result.grad_fn.next_functions}")
print(f"two_x_squared 的前一个函数: {two_x_squared.grad_fn.next_functions}")
print()

# 反向传播
print(f"🔄 步骤3: 反向传播")
print(f"计算 d(2x²+1)/dx 在 x={x.item()} 处的值")
print(f"理论结果: d(2x²+1)/dx = 4x = 4×{x.item()} = {4*x.item()}")

final_result.backward()
print(f"自动求导结果: {x.grad}")
print(f"✅ 验证通过: 理论值 = 自动求导值")

### 2.1 可视化计算图

In [ ]:
def visualize_computational_graph():
    """可视化一个简单的计算图过程"""
    print("🎨 计算图可视化")
    print("=" * 30)
    
    # 重新创建变量（清除之前的梯度）
    x = torch.tensor(3.0, requires_grad=True)
    
    print("📈 前向传播过程:")
    print("")
    print("    x = 3.0")
    print("    │")
    print("    ▼ (** 2)")
    
    y = x ** 2
    print(f"    y = x² = {y.item()}")
    print("    │")
    print("    ▼ (* 2)")
    
    z = 2 * y
    print(f"    z = 2y = {z.item()}")
    print("    │")
    print("    ▼ (+ 1)")
    
    w = z + 1
    print(f"    w = z + 1 = {w.item()}")
    print()
    
    print("📉 反向传播过程:")
    print("")
    w.backward()
    
    print("    dw/dw = 1")
    print("    ▲")
    print("    │ dw/dz = dw/dw × d(z+1)/dz = 1 × 1 = 1")
    print("    ▲")
    print("    │ dw/dy = dw/dz × d(2y)/dy = 1 × 2 = 2")
    print("    ▲")
    print("    │ dw/dx = dw/dy × d(x²)/dx = 2 × 2x = 2 × 2×3 = 12")
    print(f"    最终梯度: {x.grad}")
    
    return x.grad

gradient = visualize_computational_graph()
print(f"\n💡 这就是链式法则的自动应用！")

## ⚡ 3. 梯度的计算、存储和管理

理解梯度如何计算、存储和管理是掌握自动求导的关键。

In [ ]:
print("⚡ 梯度管理详解")
print("=" * 40)

# 1. 梯度的累积特性
print("📈 梯度累积演示:")
x = torch.tensor(2.0, requires_grad=True)

# 第一次计算
y1 = x ** 2
y1.backward()
print(f"第一次: y1 = x², x.grad = {x.grad}")

# 第二次计算（不清零梯度）
y2 = x ** 3
y2.backward()
print(f"第二次: y2 = x³, x.grad = {x.grad}  # 梯度累积了！")
print(f"解释: 第一次梯度(2x=4) + 第二次梯度(3x²=12) = {4 + 12}")
print()

# 2. 梯度清零 - 非常重要！
print("🧹 梯度清零:")
x.grad.zero_()  # 就地清零
# 或者使用: x.grad = None
print(f"清零后 x.grad = {x.grad}")

# 重新计算
y3 = x ** 3
y3.backward()
print(f"重新计算: y3 = x³, x.grad = {x.grad}  # 现在是正确的值")
print()

# 3. 标量vs向量的反向传播
print("📊 向量梯度计算:")
x_vec = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y_vec = x_vec ** 2  # [1, 4, 9]

# 对于向量，需要提供gradient参数
gradient_weights = torch.tensor([1.0, 1.0, 1.0])  # 每个元素的权重
y_vec.backward(gradient_weights)

print(f"x_vec = {x_vec.data}")
print(f"y_vec = x_vec² = {y_vec.data}")
print(f"梯度 dy/dx = 2x = {x_vec.grad}")
print(f"验证: [2×1, 2×2, 2×3] = [2, 4, 6] ✓")

### 3.1 梯度计算的高级特性

In [ ]:
print("🚀 高级梯度特性")
print("=" * 30)

# 1. retain_graph：保留计算图
print("🔄 retain_graph 使用:")
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
z = y ** 2  # z = x⁴

# 第一次反向传播，保留图
z.backward(retain_graph=True)
print(f"第一次反向传播 dz/dx = {x.grad}  # 4x³ = 4×8 = 32")

# 第二次反向传播（如果不保留图，这里会出错）
x.grad.zero_()  # 清零梯度
z.backward()  # 再次计算（因为图被保留了）
print(f"第二次反向传播 dz/dx = {x.grad}  # 结果相同")
print()

# 2. create_graph：创建梯度的梯度（二阶导数）
print("📈 二阶导数计算:")
x = torch.tensor(2.0, requires_grad=True)
y = x ** 3  # y = x³

# 计算一阶导数
grad_first = torch.autograd.grad(y, x, create_graph=True)[0]
print(f"一阶导数 dy/dx = 3x² = {grad_first}")

# 计算二阶导数
grad_second = torch.autograd.grad(grad_first, x)[0]
print(f"二阶导数 d²y/dx² = 6x = {grad_second}")
print(f"验证: 6×{x.item()} = {6*x.item()} ✓")
print()

# 3. torch.autograd.grad vs .backward()
print("🔧 torch.autograd.grad vs .backward():")
x = torch.tensor(3.0, requires_grad=True)
y = x ** 2

# 方法1：使用 .backward()（修改 x.grad）
y.backward()
print(f"方法1 (.backward): x.grad = {x.grad}")

# 方法2：使用 torch.autograd.grad（返回梯度值）
x.grad = None  # 清空梯度
y = x ** 2  # 重新计算（需要重新构建图）
grad_value = torch.autograd.grad(y, x)[0]
print(f"方法2 (autograd.grad): 返回值 = {grad_value}")
print(f"方法2 不修改 x.grad: {x.grad}")

print("\n💡 两种方法的区别:")
print("- .backward(): 将梯度存储在 .grad 属性中")
print("- autograd.grad(): 返回梯度值，不修改 .grad 属性")

## 🎯 4. 实际应用：线性回归的梯度下降

让我们用自动求导来实现一个完整的线性回归训练过程！

In [ ]:
print("🎯 实战：用自动求导实现线性回归")
print("=" * 50)

# 1. 生成模拟数据
# 真实关系：y = 2x + 1 + noise
torch.manual_seed(42)
n_samples = 100
true_w = 2.0  # 真实斜率
true_b = 1.0  # 真实截距

# 生成数据
X = torch.randn(n_samples, 1)  # 输入特征
noise = torch.randn(n_samples, 1) * 0.3  # 噪声
y_true = true_w * X + true_b + noise  # 真实标签

print(f"📊 数据生成完成:")
print(f"样本数量: {n_samples}")
print(f"真实参数: w = {true_w}, b = {true_b}")
print(f"输入范围: [{X.min():.2f}, {X.max():.2f}]")
print(f"输出范围: [{y_true.min():.2f}, {y_true.max():.2f}]")
print()

# 2. 初始化模型参数（需要梯度！）
w = torch.randn(1, 1, requires_grad=True)  # 权重
b = torch.randn(1, requires_grad=True)     # 偏置

print(f"🎲 初始参数:")
print(f"初始 w = {w.item():.4f}")
print(f"初始 b = {b.item():.4f}")
print()

# 3. 定义前向传播和损失函数
def forward(X, w, b):
    """线性回归前向传播"""
    return X @ w + b  # 矩阵乘法 + 广播

def mse_loss(y_pred, y_true):
    """均方误差损失函数"""
    return ((y_pred - y_true) ** 2).mean()

# 4. 训练循环
learning_rate = 0.1
num_epochs = 100
losses = []  # 记录损失值

print(f"🚀 开始训练:")
print(f"学习率: {learning_rate}")
print(f"训练轮数: {num_epochs}")
print("=" * 30)

for epoch in range(num_epochs):
    # 前向传播
    y_pred = forward(X, w, b)
    loss = mse_loss(y_pred, y_true)
    
    # 反向传播
    loss.backward()  # 计算梯度
    
    # 参数更新（梯度下降）
    with torch.no_grad():  # 更新参数时不需要计算梯度
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad
    
    # 清零梯度（重要！）
    w.grad.zero_()
    b.grad.zero_()
    
    # 记录损失
    losses.append(loss.item())
    
    # 打印进度
    if (epoch + 1) % 20 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:3d}: Loss = {loss.item():.6f}, w = {w.item():.4f}, b = {b.item():.4f}")

print("=" * 30)
print(f"✅ 训练完成!")
print(f"最终参数: w = {w.item():.4f}, b = {b.item():.4f}")
print(f"真实参数: w = {true_w:.4f}, b = {true_b:.4f}")
print(f"参数误差: w误差 = {abs(w.item() - true_w):.4f}, b误差 = {abs(b.item() - true_b):.4f}")

### 4.1 可视化训练过程

In [ ]:
# 可视化训练过程
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 损失曲线
ax1.plot(losses, 'b-', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('训练损失曲线')
ax1.grid(True, alpha=0.3)

# 数据和拟合结果
with torch.no_grad():
    # 创建测试数据用于绘制直线
    x_test = torch.linspace(X.min(), X.max(), 100).unsqueeze(1)
    y_pred_test = forward(x_test, w, b)
    y_true_test = true_w * x_test + true_b

# 散点图和拟合直线
ax2.scatter(X.numpy(), y_true.numpy(), alpha=0.6, label='真实数据', s=20)
ax2.plot(x_test.numpy(), y_pred_test.numpy(), 'r-', 
         label=f'学习到的直线 (w={w.item():.2f}, b={b.item():.2f})', linewidth=2)
ax2.plot(x_test.numpy(), y_true_test.numpy(), 'g--', 
         label=f'真实直线 (w={true_w:.2f}, b={true_b:.2f})', linewidth=2)
ax2.set_xlabel('X')
ax2.set_ylabel('y')
ax2.set_title('线性回归拟合结果')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎉 太棒了！自动求导帮我们完美地学习到了数据的潜在规律！")

## 🚨 5. 常见陷阱和注意事项

在使用自动求导时，有一些常见的陷阱需要注意：

In [ ]:
print("🚨 常见陷阱和解决方案")
print("=" * 40)

# 陷阱1：忘记清零梯度
print("⚠️ 陷阱1: 忘记清零梯度")
x = torch.tensor(2.0, requires_grad=True)

for i in range(3):
    y = x ** 2
    y.backward()
    print(f"第{i+1}次: x.grad = {x.grad}  # 梯度累积了！")
    # 正确做法：每次都要清零
    # x.grad.zero_()  # 取消注释以修复

print("💡 解决方案: 每次backward()前调用 x.grad.zero_()\n")

# 陷阱2：试图对非标量进行backward
print("⚠️ 陷阱2: 对非标量进行backward")
x = torch.tensor([1.0, 2.0], requires_grad=True)
y = x ** 2  # 结果是向量
try:
    y.backward()  # 这会出错！
except RuntimeError as e:
    print(f"错误: {str(e)[:50]}...")
    print("💡 解决方案1: 先求和再backward")
    y.sum().backward()
    print(f"求和后backward成功: x.grad = {x.grad}")
    
# 或者提供gradient参数
x.grad.zero_()
y = x ** 2
gradient_weights = torch.tensor([1.0, 1.0])
y.backward(gradient_weights)
print(f"💡 解决方案2: 提供gradient参数: x.grad = {x.grad}\n")

# 陷阱3：in-place操作破坏计算图
print("⚠️ 陷阱3: in-place操作破坏计算图")
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
try:
    y += 1  # in-place操作，等价于 y.add_(1)
    y.backward()
except RuntimeError as e:
    print(f"错误: in-place操作可能破坏计算图")
    print("💡 解决方案: 使用非in-place操作")
    x = torch.tensor(2.0, requires_grad=True)
    y = x ** 2
    z = y + 1  # 非in-place操作
    z.backward()
    print(f"修复后: x.grad = {x.grad}\n")

# 陷阱4：在no_grad上下文中意外关闭梯度
print("⚠️ 陷阱4: 意外关闭梯度计算")
x = torch.tensor(2.0, requires_grad=True)
with torch.no_grad():
    y = x ** 2  # 在no_grad中计算
print(f"y.requires_grad = {y.requires_grad}  # False！梯度被关闭了")
print("💡 解决方案: 确保需要梯度的计算在no_grad之外\n")

# 陷阱5：重复使用中间变量
print("⚠️ 陷阱5: 重复backward同一个图")
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
y.backward()  # 第一次backward
try:
    y.backward()  # 第二次backward，图已经被释放！
except RuntimeError as e:
    print(f"错误: 计算图已被释放")
    print("💡 解决方案: 使用 retain_graph=True 或重新计算")
    x = torch.tensor(2.0, requires_grad=True)
    y = x ** 2
    y.backward(retain_graph=True)  # 保留图
    x.grad.zero_()  # 清零梯度
    y.backward()  # 现在可以再次backward
    print(f"修复后: x.grad = {x.grad}")

## 🔧 6. 自动求导的高级应用

让我们探索一些高级应用场景：

In [ ]:
print("🔧 高级应用场景")
print("=" * 30)

# 1. 计算雅可比矩阵 (Jacobian)
print("📊 计算雅可比矩阵:")

def compute_jacobian(func, inputs):
    """计算函数的雅可比矩阵"""
    outputs = func(inputs)
    jacobian = torch.zeros(outputs.size(0), inputs.size(0))
    
    for i in range(outputs.size(0)):
        # 为每个输出分量计算梯度
        grad_outputs = torch.zeros_like(outputs)
        grad_outputs[i] = 1.0
        
        inputs.grad = None  # 清零梯度
        grad_inputs = torch.autograd.grad(outputs, inputs, 
                                        grad_outputs=grad_outputs,
                                        retain_graph=True)[0]
        jacobian[i] = grad_inputs
    
    return jacobian

# 示例：f(x) = [x1², x1*x2, x2²]
def vector_function(x):
    x1, x2 = x[0], x[1]
    return torch.stack([x1**2, x1*x2, x2**2])

x = torch.tensor([2.0, 3.0], requires_grad=True)
jacobian = compute_jacobian(vector_function, x)

print(f"输入: x = {x.data}")
print(f"函数: f(x) = [x1², x1*x2, x2²]")
print(f"雅可比矩阵:\n{jacobian}")
print(f"理论值:")
print(f"df1/dx1 = 2*x1 = {2*x[0]}, df1/dx2 = 0")
print(f"df2/dx1 = x2 = {x[1]}, df2/dx2 = x1 = {x[0]}")
print(f"df3/dx1 = 0, df3/dx2 = 2*x2 = {2*x[1]}\n")

# 2. 梯度检查 (Gradient Checking)
print("🔍 梯度检查:")

def numerical_gradient(func, x, eps=1e-7):
    """数值方法计算梯度"""
    grad = torch.zeros_like(x)
    for i in range(x.size(0)):
        x_plus = x.clone()
        x_minus = x.clone()
        x_plus[i] += eps
        x_minus[i] -= eps
        
        grad[i] = (func(x_plus) - func(x_minus)) / (2 * eps)
    return grad

def test_function(x):
    return (x ** 3).sum()  # f(x) = x1³ + x2³

x = torch.tensor([2.0, 3.0], requires_grad=True)

# 自动求导
y = test_function(x)
y.backward()
auto_grad = x.grad.clone()

# 数值求导
x_no_grad = torch.tensor([2.0, 3.0])  # 不需要梯度的版本
numerical_grad = numerical_gradient(test_function, x_no_grad)

print(f"自动求导梯度: {auto_grad}")
print(f"数值求导梯度: {numerical_grad}")
print(f"差异: {torch.abs(auto_grad - numerical_grad)}")
print(f"相对误差: {torch.abs(auto_grad - numerical_grad) / torch.abs(auto_grad)}")
print("✅ 梯度检查通过！(差异极小)\n")

# 3. 条件梯度计算
print("🎭 条件梯度计算:")

x = torch.tensor(2.0, requires_grad=True)
condition = x > 1.0

if condition:
    y = x ** 2
else:
    y = x ** 3

y.backward()
print(f"当 x = {x.item()} > 1 时:")
print(f"使用函数 y = x²")
print(f"梯度 dy/dx = 2x = {x.grad}")

# 测试另一个条件
x = torch.tensor(0.5, requires_grad=True)
if x > 1.0:
    y = x ** 2
else:
    y = x ** 3

y.backward()
print(f"\n当 x = {x.item()} ≤ 1 时:")
print(f"使用函数 y = x³")
print(f"梯度 dy/dx = 3x² = {x.grad}")

## 🎯 7. 实战：多层感知机的反向传播

让我们手动实现一个简单的神经网络，深入理解反向传播：

In [ ]:
print("🧠 实战：手动实现神经网络反向传播")
print("=" * 50)

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        """简单的两层神经网络
        
        网络结构: input → hidden → output
        激活函数: ReLU (隐藏层), 无激活 (输出层)
        """
        # 初始化权重（需要梯度！）
        self.W1 = torch.randn(input_size, hidden_size, requires_grad=True) * 0.5
        self.b1 = torch.zeros(hidden_size, requires_grad=True)
        
        self.W2 = torch.randn(hidden_size, output_size, requires_grad=True) * 0.5
        self.b2 = torch.zeros(output_size, requires_grad=True)
        
        print(f"🏗️ 网络结构: {input_size} → {hidden_size} → {output_size}")
        print(f"参数数量: {self.count_parameters()}")
    
    def forward(self, x):
        """前向传播"""
        # 第一层: input → hidden
        z1 = x @ self.W1 + self.b1  # 线性变换
        a1 = torch.relu(z1)         # ReLU激活
        
        # 第二层: hidden → output
        z2 = a1 @ self.W2 + self.b2  # 线性变换
        
        return z2
    
    def count_parameters(self):
        """统计参数数量"""
        total = 0
        for param in [self.W1, self.b1, self.W2, self.b2]:
            total += param.numel()
        return total
    
    def zero_grad(self):
        """清零所有参数的梯度"""
        for param in [self.W1, self.b1, self.W2, self.b2]:
            if param.grad is not None:
                param.grad.zero_()
    
    def get_parameters(self):
        """获取所有参数"""
        return [self.W1, self.b1, self.W2, self.b2]

# 创建网络和数据
torch.manual_seed(42)
net = SimpleNeuralNetwork(input_size=3, hidden_size=5, output_size=2)

# 生成一批样本数据
batch_size = 4
X = torch.randn(batch_size, 3)  # 4个样本，每个3维特征
y_true = torch.randn(batch_size, 2)  # 4个样本，每个2维输出

print(f"\n📊 数据信息:")
print(f"输入形状: {X.shape}")
print(f"目标形状: {y_true.shape}")
print()

# 训练一个step，详细观察梯度
print(f"🔄 详细的一步训练过程:")
print("-" * 30)

# 1. 前向传播
print("1️⃣ 前向传播:")
y_pred = net.forward(X)
print(f"预测输出形状: {y_pred.shape}")
print(f"预测值范围: [{y_pred.min():.3f}, {y_pred.max():.3f}]")

# 2. 计算损失
print("\n2️⃣ 计算损失:")
loss = torch.nn.functional.mse_loss(y_pred, y_true)
print(f"MSE损失: {loss.item():.6f}")

# 3. 反向传播
print("\n3️⃣ 反向传播:")
print("计算所有参数的梯度...")
loss.backward()

# 检查梯度
print("\n📊 梯度统计:")
param_names = ['W1', 'b1', 'W2', 'b2']
for name, param in zip(param_names, net.get_parameters()):
    if param.grad is not None:
        grad_norm = param.grad.norm().item()
        grad_mean = param.grad.mean().item()
        print(f"{name}: 梯度范数={grad_norm:.6f}, 梯度均值={grad_mean:.6f}")
    else:
        print(f"{name}: 梯度为None")

# 4. 参数更新
print("\n4️⃣ 参数更新:")
learning_rate = 0.01
with torch.no_grad():
    for param in net.get_parameters():
        param -= learning_rate * param.grad
print(f"学习率: {learning_rate}")
print("参数更新完成")

# 5. 清零梯度
print("\n5️⃣ 清零梯度:")
net.zero_grad()
print("梯度清零完成")

print("\n🎉 一步完整的训练循环完成！")
print("这就是深度学习训练的核心过程：前向→损失→反向→更新→清零")

### 7.1 可视化梯度流

In [ ]:
def visualize_gradient_flow(net, X, y_true, num_steps=10):
    """可视化训练过程中的梯度变化"""
    losses = []
    w1_grads = []
    w2_grads = []
    
    print("📈 梯度流可视化 (前10步训练)")
    print("-" * 40)
    
    for step in range(num_steps):
        # 前向传播
        y_pred = net.forward(X)
        loss = torch.nn.functional.mse_loss(y_pred, y_true)
        
        # 反向传播
        net.zero_grad()
        loss.backward()
        
        # 记录数据
        losses.append(loss.item())
        w1_grads.append(net.W1.grad.norm().item())
        w2_grads.append(net.W2.grad.norm().item())
        
        # 参数更新
        with torch.no_grad():
            for param in net.get_parameters():
                param -= 0.01 * param.grad
        
        if step % 3 == 0:
            print(f"Step {step:2d}: Loss={loss.item():.4f}, "
                  f"W1_grad_norm={w1_grads[-1]:.4f}, "
                  f"W2_grad_norm={w2_grads[-1]:.4f}")
    
    # 绘制图像
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # 损失曲线
    ax1.plot(losses, 'b-', marker='o', markersize=4)
    ax1.set_xlabel('Training Step')
    ax1.set_ylabel('Loss')
    ax1.set_title('训练损失变化')
    ax1.grid(True, alpha=0.3)
    
    # 梯度范数
    ax2.plot(w1_grads, 'r-', marker='s', markersize=4, label='W1 梯度范数')
    ax2.plot(w2_grads, 'g-', marker='^', markersize=4, label='W2 梯度范数')
    ax2.set_xlabel('Training Step')
    ax2.set_ylabel('Gradient Norm')
    ax2.set_title('梯度范数变化')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return losses, w1_grads, w2_grads

# 运行可视化
losses, w1_grads, w2_grads = visualize_gradient_flow(net, X, y_true)

print(f"\n📊 观察结果:")
print(f"• 损失从 {losses[0]:.4f} 降至 {losses[-1]:.4f}")
print(f"• W1层梯度范数: {w1_grads[0]:.4f} → {w1_grads[-1]:.4f}")
print(f"• W2层梯度范数: {w2_grads[0]:.4f} → {w2_grads[-1]:.4f}")
print(f"✅ 梯度正常流动，网络在学习！")

## 📝 8. 自动求导原理深度解析

让我们深入理解自动求导的底层原理：

In [ ]:
print("🔬 自动求导原理深度解析")
print("=" * 40)

# 1. 链式法则的逐步展示
print("⛓️ 链式法则详细展示:")
print("函数: f(x) = sin(x²)")
print("分解: u = x², v = sin(u), f = v")
print("链式法则: df/dx = (df/dv) × (dv/du) × (du/dx)")

x = torch.tensor(1.0, requires_grad=True)

# 逐步计算，保存中间结果
u = x ** 2
print(f"\n中间步骤:")
print(f"x = {x.item():.4f}")
print(f"u = x² = {u.item():.4f}")

v = torch.sin(u)
print(f"v = sin(u) = {v.item():.4f}")

# 反向传播
v.backward()
print(f"\n梯度计算:")
print(f"df/dx = {x.grad:.6f}")

# 手动验证
manual_grad = torch.cos(u) * 2 * x
print(f"手动计算: cos({u.item():.4f}) × 2 × {x.item()} = {manual_grad.item():.6f}")
print(f"误差: {abs(x.grad.item() - manual_grad.item()):.10f}\n")

# 2. 不同运算的局部梯度
print("🧮 常见运算的局部梯度:")
operations = [
    ("加法", "z = x + y", "dz/dx = 1, dz/dy = 1"),
    ("乘法", "z = x * y", "dz/dx = y, dz/dy = x"),
    ("指数", "z = x^n", "dz/dx = n * x^(n-1)"),
    ("对数", "z = log(x)", "dz/dx = 1/x"),
    ("ReLU", "z = max(0,x)", "dz/dx = 1 if x>0 else 0"),
    ("Sigmoid", "z = 1/(1+e^(-x))", "dz/dx = z*(1-z)")
]

for name, func, grad in operations:
    print(f"{name:8s}: {func:15s} → {grad}")

print("\n💡 自动求导就是自动应用这些规则！")

# 3. 计算图的内存管理
print("\n💾 计算图的内存管理:")

x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
z = y * 3

print(f"创建计算图: x → y → z")
print(f"z.grad_fn: {z.grad_fn}")
print(f"y.grad_fn: {y.grad_fn}")
print(f"x.grad_fn: {x.grad_fn}")

# 检查引用计数
import sys
print(f"\nPython引用计数:")
print(f"x引用数: {sys.getrefcount(x)}")
print(f"y引用数: {sys.getrefcount(y)}")
print(f"z引用数: {sys.getrefcount(z)}")

# backward后图被释放
z.backward()
print(f"\nbackward后:")
print(f"z.grad_fn: {z.grad_fn}  # 图被释放")

# 4. 动态计算图 vs 静态计算图
print("\n🔄 动态计算图的优势:")

for i in range(3):
    x = torch.tensor(float(i+1), requires_grad=True)
    
    if i % 2 == 0:
        y = x ** 2  # 偶数次迭代
    else:
        y = x ** 3  # 奇数次迭代
    
    y.backward()
    print(f"迭代{i}: x={x.item()}, 函数=x^{2 if i%2==0 else 3}, 梯度={x.grad.item()}")

print("💡 每次前向传播都可以使用不同的计算图！")

## 🎓 总结与关键要点

### 🎯 核心概念回顾

1. **自动求导是深度学习的基石** - 让我们无需手动推导复杂的梯度公式
2. **计算图记录运算历史** - 前向传播构建图，反向传播计算梯度
3. **链式法则自动应用** - PyTorch自动处理复合函数的梯度计算
4. **动态图的灵活性** - 每次前向传播都可以使用不同的计算逻辑

### 💡 实用技巧总结

- ✅ **始终记住清零梯度**: `tensor.grad.zero_()` 或 `tensor.grad = None`
- ✅ **合理使用 `requires_grad`**: 只为需要梯度的张量设置
- ✅ **避免 in-place 操作**: 使用 `y = x + 1` 而不是 `x += 1`
- ✅ **理解 `no_grad` 上下文**: 推理时关闭梯度计算节省内存
- ✅ **使用梯度检查验证**: 数值梯度 vs 自动梯度

### 🚀 下一步学习建议

1. **优化器 (Optimizers)** - SGD, Adam, RMSprop等
2. **损失函数 (Loss Functions)** - 各种损失函数的设计原理
3. **神经网络模块 (nn.Module)** - 构建复杂模型的基础
4. **高级自动求导** - 高阶导数、雅可比矩阵等

---

In [ ]:
print("🏆 恭喜完成自动求导机制学习！")
print("=" * 40)

# 最终测试：综合应用
print("🎯 综合测试: 实现一个简单的优化问题")
print("问题: 找到函数 f(x,y) = (x-3)² + (y-2)² 的最小值")
print("理论答案: x=3, y=2时达到最小值0")

# 使用梯度下降求解
x = torch.tensor(0.0, requires_grad=True)  # 起始点 (0, 0)
y = torch.tensor(0.0, requires_grad=True)

learning_rate = 0.1
for step in range(50):
    # 目标函数
    f = (x - 3)**2 + (y - 2)**2
    
    # 反向传播
    f.backward()
    
    # 梯度下降更新
    with torch.no_grad():
        x -= learning_rate * x.grad
        y -= learning_rate * y.grad
    
    # 清零梯度
    x.grad.zero_()
    y.grad.zero_()
    
    if step % 10 == 0:
        print(f"Step {step:2d}: f={f.item():.6f}, x={x.item():.4f}, y={y.item():.4f}")

print(f"\n🎉 最终结果:")
print(f"找到的最优解: x = {x.item():.4f}, y = {y.item():.4f}")
print(f"理论最优解:   x = 3.0000, y = 2.0000")
print(f"最小函数值:   f = {f.item():.8f}")

error = ((x.item()-3)**2 + (y.item()-2)**2)**0.5
print(f"误差距离:     {error:.6f}")

if error < 0.01:
    print("✅ 优化成功！误差在可接受范围内")
else:
    print("❌ 需要更多训练步数或调整学习率")

print("\n💪 你已经掌握了自动求导的核心原理和应用！")
print("现在可以开始学习更高级的深度学习概念了。")